# Solutions

Here are are sample solutions to the exercises of the two sections.

#### Table of Contents
- [Section 1 &ndash; Getting Started with Pandas](#Section-1)
- [Section 2 &ndash; Data Wrangling](#Section-2)

--- 

## Section 1

### Exercise 1.1
##### Create a DataFrame by reading in the `2019_Yellow_Taxi_Trip_Data.csv` file. Examine the first 5 rows.

In [ ]:
import pandas as pd

# Read the data from the CSV file
taxis = pd.read_csv('../data/2019_Yellow_Taxi_Trip_Data.csv')

# Display the first few rows of the data
taxis.head()

### Exercise 1.2
##### Find the dimensions (number of rows and number of columns) in the data.

In [ ]:
# Display the shape of the data
taxis.shape

### Exercise 1.3
##### Using the data in the `2019_Yellow_Taxi_Trip_Data.csv` file, calculate summary statistics for the `fare_amount`, `tip_amount`, `tolls_amount`, and `total_amount` columns.

In [ ]:
import pandas as pd

# Read the data from the CSV file
taxis = pd.read_csv('../data/2019_Yellow_Taxi_Trip_Data.csv')

# Describe a subset of the columns
taxis[['fare_amount', 'tip_amount', 'tolls_amount', 'total_amount']].describe()

### Exercise 1.4
##### Get the `fare_amount`, `tip_amount`, `tolls_amount`, and `total_amount` for the longest trip by distance (`trip_distance`).

In [ ]:
taxis.loc[
    taxis['trip_distance'].idxmax(), # Get the index of the row with the maximum trip distance
    ['fare_amount', 'tip_amount', 'tolls_amount', 'total_amount'] # Get the subset of columns
]

---

## Section 2

### Exercise 2.1
##### Read in the meteorite data from the `Meteorite_Landings.csv` file, rename the `mass (g)` column to `mass`, and drop all the latitude and longitude columns. Sort the result by mass in descending order.

In [ ]:
import pandas as pd

# Read the data from the CSV file
meteorites = pd.read_csv('../data/Meteorite_Landings.csv')

# Rename the column
meteorites = meteorites.rename(columns={'mass (g)': 'mass'})

# Drop irrelevant columns
meteorites = meteorites.drop(columns=meteorites.columns[-3:])

# Sort the data by mass
meteorites.sort_values('mass', ascending=False)

# Display the first few rows of the data
meteorites.head()

### Exercise 2.2
##### Using the meteorite data from the `Meteorite_Landings.csv` file, update the `year` column to only contain the year, convert it to a numeric data type, and create a new column indicating whether the meteorite was observed falling before 1970. Set the index to the `id` column and extract all the rows with IDs between 10,036 and 10,040 (inclusive) with `loc[]`.

###### **Hint 1**: Use `year.str.slice()` to grab a substring.

###### **Hint 2**: Make sure to sort the index before using `loc[]` to select the range.

###### **Bonus**: There's a data entry error in the `year` column. Can you find it? (Don't spend too much time on this.)

In [ ]:
import pandas as pd

# Load the CSV file
meteorites = pd.read_csv('../data/Meteorite_Landings.csv')

# Convert 'year' to numeric by extracting the last 4 characters from the 'year' string
meteorites['year'] = pd.to_numeric(meteorites['year'].str.slice(6, 10))

# Create the 'pre_1970' column indicating if the meteorite fell before 1970
meteorites['pre_1970'] = (meteorites['fall'] == 'Fell') & (meteorites['year'] < 1970)

# Set 'id' as the index
meteorites = meteorites.set_index('id')

# Sort by index
meteorites = meteorites.sort_index()

# Extract IDs between 10,036 and 10,040
meteorites.loc[10_036:10_040]

**Note**: The `pd.to_datetime()` function is another option here; however, it will only be able to convert dates within the supported bounds (between `pd.Timestamp.min` and `pd.Timestamp.max`), which will cause some entries that do have a year to be marked as not having one. More information can be found in the pandas documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html). For reference, this is how the conversion can be done:

```python
pd.to_datetime(
    meteorites.year,
    errors='coerce',  # anything that can't be converted will be NaT (null)
    format='%m/%d/%Y %I:%M:%S %p'  # the format the datetimes are currently in
)
```

##### **Bonus**: There's a data entry error in the `year` column. Can you find it?

In [ ]:
meteorites.year.describe()

There's a meteorite that was reportedly found in the future:

In [ ]:
meteorites.query(f'year > {pd.Timestamp("today").year}')

Oops! This meteorite actually was found in 2010 (more information [here](https://www.lpi.usra.edu/meteor/metbull.php?code=57150)).

### Exercise 2.3
##### Using the meteorite data from the `Meteorite_Landings.csv` file, create a pivot table that shows both the number of meteorites and the 95th percentile of meteorite mass for those that were found versus observed falling per year from 2005 through 2009 (inclusive). Hint: Be sure to convert the `year` column to a number as we did in the previous exercise.

In [ ]:
import pandas as pd

# Load the CSV file
meteorites = pd.read_csv('../data/Meteorite_Landings.csv')

# Convert 'year' to numeric by extracting the last 4 characters from the 'year' string
meteorites['year'] = pd.to_numeric(meteorites['year'].str.slice(6, 10))

# Filter rows where 'year' is between 2005 and 2009
filtered_meteorites = meteorites[(meteorites['year']>=2005) & (meteorites['year']<=2009)]

# Create a pivot table with 'year' as the index, 'fall' as the columns, 
# and count and 95th percentile as aggregation functions for 'mass (g)'
pivot_table = filtered_meteorites.pivot_table(
    index='year',
    columns='fall',
    values='mass (g)',
    aggfunc={
        'mass (g)': ['count', lambda x: x.quantile(0.95)]
    }
)

# Rename the 'lambda' function column to '95th percentile'
pivot_table = pivot_table.rename(columns={'<lambda_0>': '95th percentile'})

pivot_table

### Exercise 2.4
##### Using the meteorite data from the `Meteorite_Landings.csv` file, compare summary statistics of the mass column for the meteorites that were found versus observed falling.

In [ ]:
import pandas as pd

# Load the CSV file
meteorites = pd.read_csv('../data/Meteorite_Landings.csv')

# Group by fall and describe the 'mass (g)' column
meteorites.groupby('fall')['mass (g)'].describe()